In [ ]:


# Data Analysis
import pandas as pd
import polars as pl
import numpy as np

# Visualization
import seaborn as sns
import matplotlib.pyplot as plt


# Stats & ML
from scipy.stats import  zscore
from sklearn.ensemble import IsolationForest



pl.Config.set_tbl_rows(-1)
%matplotlib inline

## Data Loading

In [ ]:
try:
    df = pl.read_csv('../exports/imputed_water_meters_v2.csv', infer_schema_length=100000)
    print(f"Data loaded successfully")
except Exception as e:
    print(f"An error occurred while loading the data: {e}")

# Exploratory Data Analysis

In [ ]:
df.head()

#### What is the total available data?

In [ ]:
df.shape[0]

In [ ]:
df['DEVICE_ID'].n_unique()

#### What are some descriptive statistics about the data ?

In [ ]:
df.describe()

In [ ]:
df.head()

#### Convert date string to Datetime

In [ ]:
df = df.with_columns(
    pl.col("DATE").cast(pl.Date)
)

In [ ]:
df = df.rename({
            "DENORMALIZED_LIN_REG_IMPUTED":"LIN_REG_IMPUTED",
            #     "DENORMALIZED_SAITS_IMPUTED":"SAITS_IMPUTED",
                "DENORMALIZED_KNN_IMPUTED":"KNN_IMPUTED",
                "CUMMULATIVE_CONSUMPTION_COPY":"CUMMULATIVE_CONSUMPTION"
               })

In [ ]:
df.head()

#### Daily Consumption - for Validity

In [ ]:
imputed_columns = [
    "MEAN_IMPUTED", "MEDIAN_IMPUTED", "FFILL_IMPUTED", "BFILL_IMPUTED",
    "LINEAR_IMPUTED", "CUBIC_IMPUTED", "KNN_IMPUTED",
    "LIN_REG_IMPUTED", # "SAITS_IMPUTED"
]

# Loop through the imputed columns and calculate daily differences
daily_diff_exprs = [
    (pl.col(col) - pl.col(col).shift(1))
    .over("DEVICE_ID")
    .alias(f"DAILY_DIFF_{col}")
    for col in imputed_columns
]

# Add all the daily difference columns to the DataFrame
df_with_diffs = df.with_columns(daily_diff_exprs)

In [ ]:
df_with_diffs.head()

In [ ]:
daily_consumption = df_with_diffs

# Data Quality Post Assessment

### Validity

In [ ]:
# Define the validity check expressions for each daily diff
validity_checks = [
    pl.when(
        (pl.col("CUMMULATIVE_CONSUMPTION") == 0) | (pl.col(f"DAILY_DIFF_{col}") < 0)
    )
    .then(0)  # Invalid
    .otherwise(1)  # Valid
    .alias(f"VALIDITY_{col}")
    for col in imputed_columns
]

# Apply the validity checks to the DataFrame
daily_consumption = daily_consumption.with_columns(validity_checks)

daily_consumption.head()


In [ ]:
daily_consumption['VALIDITY_MEAN_IMPUTED'].value_counts()

In [ ]:
daily_consumption['VALIDITY_MEDIAN_IMPUTED'].value_counts()

In [ ]:
daily_consumption['VALIDITY_FFILL_IMPUTED'].value_counts()

In [ ]:
daily_consumption['VALIDITY_BFILL_IMPUTED'].value_counts()

In [ ]:
daily_consumption['VALIDITY_LINEAR_IMPUTED'].value_counts()

In [ ]:
daily_consumption['VALIDITY_CUBIC_IMPUTED'].value_counts()

In [ ]:
daily_consumption['VALIDITY_KNN_IMPUTED'].value_counts()

In [ ]:
daily_consumption['VALIDITY_LIN_REG_IMPUTED'].value_counts()

In [ ]:
# daily_consumption['VALIDITY_SAITS_IMPUTED'].value_counts()

In [ ]:
df.shape[0]

### Accuracy

In [ ]:
# Z-score method
def detect_outliers_zscore(data, threshold=3):
    return np.abs(zscore(data)) > threshold

# IQR method
def detect_outliers_iqr(data):
    q1 = np.percentile(data, 25)
    q3 = np.percentile(data, 75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    return (data < lower_bound) | (data > upper_bound)

# MAD method
def detect_outliers_mad(data, threshold=3):
    median = np.median(data)
    mad = np.median(np.abs(data - median))
    modified_z_score = 0.6745 * (data - median) / mad
    return np.abs(modified_z_score) > threshold

# Isolation Forest method
def detect_outliers_isolation_forest(data):
    iso = IsolationForest(random_state=42,contamination='auto',)
    return iso.fit_predict(data.reshape(-1, 1)) == -1


In [ ]:
# Outlier detection workflow for a single device
def detect_outliers_per_device(device_data,target,validity_column):
    # Mark `VALIDITY=0` as outliers automatically
    device_data = device_data.with_columns(
        pl.when(pl.col(validity_column) == 0)
        .then(True)  # Automatically mark as outlier
        .otherwise(False)
        .alias(f"FINAL_OUTLIER_{target}")
    )

    # Filter out records with VALIDITY=0 before outlier detection
    valid_data = device_data.filter(pl.col(validity_column) == 1)

    # Proceed only if valid data exists
    if not valid_data.is_empty():
      
        data = valid_data[target].to_numpy()

        # Initialize empty columns for all possible outlier methods
        valid_data = valid_data.with_columns(
            pl.lit(False).alias("OUTLIER_ZSCORE"),
            pl.lit(False).alias("OUTLIER_IQR"),
            pl.lit(False).alias("OUTLIER_MAD"),
            pl.lit(False).alias("OUTLIER_ISO"),

        )

       
        iqr_outliers = detect_outliers_iqr(data)
        mad_outliers = detect_outliers_mad(data)
        iso_outliers = detect_outliers_isolation_forest(data)
        zscore_outliers = detect_outliers_zscore(data)
        valid_data = valid_data.with_columns(
            pl.Series("OUTLIER_ISO", iso_outliers),
            pl.Series("OUTLIER_MAD", mad_outliers),
            # pl.Series("OUTLIER_LOF", lof_outliers),
            pl.Series("OUTLIER_IQR", iqr_outliers),
        )
        # Combine MAD and Isolation Forest using logical AND
        final_outlier = np.logical_and(zscore_outliers, np.logical_and(iqr_outliers,np.logical_and(mad_outliers, iso_outliers)))

        # Update FINAL_OUTLIER for valid records
        valid_data = valid_data.with_columns(
            pl.Series(f"FINAL_OUTLIER_{target}", final_outlier)
        )

        # Add missing columns to invalid data with default values
        invalid_data = device_data.filter(pl.col(validity_column) == 0).with_columns(
            pl.lit(False).alias("OUTLIER_ZSCORE"),
            pl.lit(False).alias("OUTLIER_IQR"),
            pl.lit(False).alias("OUTLIER_MAD"),
            pl.lit(False).alias("OUTLIER_ISO"),
  
        )

        # Combine updated valid data with invalid data
        device_data = valid_data.vstack(invalid_data)
        
    return device_data

In [ ]:
imputed_columns = [
    "MEAN_IMPUTED", "MEDIAN_IMPUTED", "FFILL_IMPUTED", "BFILL_IMPUTED",
    "LINEAR_IMPUTED", "CUBIC_IMPUTED", "KNN_IMPUTED",
    "LIN_REG_IMPUTED", # "SAITS_IMPUTED"
]

In [ ]:
df.head()

#### Mean Imputation

In [ ]:
# Group data by DEVICE_ID and apply the workflow
result = []
for device_id, group in daily_consumption.group_by("DEVICE_ID"):
    processed_group = detect_outliers_per_device(group,"MEAN_IMPUTED","VALIDITY_MEAN_IMPUTED")
    result.append(processed_group)

# Concatenate results into a single DataFrame
mean_result_df = pl.concat(result)

In [ ]:
mean_result_df['FINAL_OUTLIER_MEAN_IMPUTED'].value_counts()

#### Median Imputation

In [ ]:
# Group data by DEVICE_ID and apply the workflow
median_result = []
for device_id, group in daily_consumption.group_by("DEVICE_ID"):
    processed_group = detect_outliers_per_device(group,"MEDIAN_IMPUTED","VALIDITY_MEDIAN_IMPUTED")
    median_result.append(processed_group)
# Concatenate results into a single DataFrame
median_result_df = pl.concat(median_result)

In [ ]:
median_result_df['FINAL_OUTLIER_MEDIAN_IMPUTED'].value_counts()

#### Forward Fill

In [ ]:
# Group data by DEVICE_ID and apply the workflow
ffill_result = []
for device_id, group in daily_consumption.group_by("DEVICE_ID"):
    processed_group = detect_outliers_per_device(group,"FFILL_IMPUTED","VALIDITY_FFILL_IMPUTED")
    ffill_result.append(processed_group)
# Concatenate results into a single DataFrame
ffill_result_df = pl.concat(ffill_result)

In [ ]:
ffill_result_df['FINAL_OUTLIER_FFILL_IMPUTED'].value_counts()

#### Backward Fill

In [ ]:
# Group data by DEVICE_ID and apply the workflow
bfill_result = []
for device_id, group in daily_consumption.group_by("DEVICE_ID"):
    processed_group = detect_outliers_per_device(group,"BFILL_IMPUTED","VALIDITY_BFILL_IMPUTED")
    bfill_result.append(processed_group)
# Concatenate results into a single DataFrame
bfill_result_df = pl.concat(bfill_result)

In [ ]:
bfill_result_df['FINAL_OUTLIER_BFILL_IMPUTED'].value_counts()

#### Cubic Interpolation

In [ ]:
# Group data by DEVICE_ID and apply the workflow
cubic_result = []
for device_id, group in daily_consumption.group_by("DEVICE_ID"):
    processed_group = detect_outliers_per_device(group,"CUBIC_IMPUTED","VALIDITY_CUBIC_IMPUTED")
    cubic_result.append(processed_group)
# Concatenate results into a single DataFrame
cubic_result_df = pl.concat(cubic_result)

In [ ]:
cubic_result_df['FINAL_OUTLIER_CUBIC_IMPUTED'].value_counts()

#### Linear Interpolation

In [ ]:
# Group data by DEVICE_ID and apply the workflow
linear_result = []
for device_id, group in daily_consumption.group_by("DEVICE_ID"):
    processed_group = detect_outliers_per_device(group,"LINEAR_IMPUTED","VALIDITY_LINEAR_IMPUTED")
    linear_result.append(processed_group)
# Concatenate results into a single DataFrame
linear_result_df = pl.concat(linear_result)

In [ ]:
linear_result_df['FINAL_OUTLIER_LINEAR_IMPUTED'].value_counts()

#### KNN

In [ ]:
# Group data by DEVICE_ID and apply the workflow
knn_result = []
for device_id, group in daily_consumption.group_by("DEVICE_ID"):
    processed_group = detect_outliers_per_device(group,"KNN_IMPUTED","VALIDITY_KNN_IMPUTED")
    knn_result.append(processed_group)
# Concatenate results into a single DataFrame
knn_result_df = pl.concat(knn_result)

In [ ]:
knn_result_df['FINAL_OUTLIER_KNN_IMPUTED'].value_counts()

#### Linear Regression

In [ ]:
# Group data by DEVICE_ID and apply the workflow
lin_reg_result = []
for device_id, group in daily_consumption.group_by("DEVICE_ID"):
    processed_group = detect_outliers_per_device(group,"LIN_REG_IMPUTED","VALIDITY_LIN_REG_IMPUTED")
    lin_reg_result.append(processed_group)
# Concatenate results into a single DataFrame
lin_reg_result_df = pl.concat(lin_reg_result)

In [ ]:
lin_reg_result_df['FINAL_OUTLIER_LIN_REG_IMPUTED'].value_counts()

#### SAITS

In [ ]:
# # Group data by DEVICE_ID and apply the workflow
# saits_reg_result = []
# for device_id, group in daily_consumption.group_by("DEVICE_ID"):
#     processed_group = detect_outliers_per_device(group,"SAITS_IMPUTED","VALIDITY_SAITS_IMPUTED")
#     saits_reg_result.append(processed_group)
# # Concatenate results into a single DataFrame
# saits_result_df = pl.concat(saits_reg_result)

In [ ]:
# saits_result_df['FINAL_OUTLIER_SAITS_IMPUTED'].value_counts()

#### Join All the results

In [ ]:
# Start with the main dataframe
merged_df = df

# Perform sequential joins for each result dataframe
merged_df = merged_df.join(mean_result_df.select(["DEVICE_ID", "DATE", "VALIDITY_MEAN_IMPUTED","FINAL_OUTLIER_MEAN_IMPUTED"]),
                           on=["DEVICE_ID", "DATE"], how="left")

merged_df = merged_df.join(median_result_df.select(["DEVICE_ID", "DATE","VALIDITY_MEDIAN_IMPUTED", "FINAL_OUTLIER_MEDIAN_IMPUTED"]),
                           on=["DEVICE_ID", "DATE"], how="left")

merged_df = merged_df.join(lin_reg_result_df.select(["DEVICE_ID", "DATE", "VALIDITY_LIN_REG_IMPUTED","FINAL_OUTLIER_LIN_REG_IMPUTED"]),
                           on=["DEVICE_ID", "DATE"], how="left")

merged_df = merged_df.join(knn_result_df.select(["DEVICE_ID", "DATE","VALIDITY_KNN_IMPUTED", "FINAL_OUTLIER_KNN_IMPUTED"]),
                           on=["DEVICE_ID", "DATE"], how="left")

merged_df = merged_df.join(linear_result_df.select(["DEVICE_ID", "DATE", "VALIDITY_LINEAR_IMPUTED","FINAL_OUTLIER_LINEAR_IMPUTED"]),
                           on=["DEVICE_ID", "DATE"], how="left")

merged_df = merged_df.join(cubic_result_df.select(["DEVICE_ID", "DATE","VALIDITY_CUBIC_IMPUTED", "FINAL_OUTLIER_CUBIC_IMPUTED"]),
                           on=["DEVICE_ID", "DATE"], how="left")

merged_df = merged_df.join(bfill_result_df.select(["DEVICE_ID", "DATE","VALIDITY_BFILL_IMPUTED", "FINAL_OUTLIER_BFILL_IMPUTED"]),
                           on=["DEVICE_ID", "DATE"], how="left")

merged_df = merged_df.join(ffill_result_df.select(["DEVICE_ID", "DATE", "VALIDITY_FFILL_IMPUTED","FINAL_OUTLIER_FFILL_IMPUTED"]),
                           on=["DEVICE_ID", "DATE"], how="left")

# merged_df = merged_df.join(saits_result_df.select(["DEVICE_ID", "DATE", "VALIDITY_SAITS_IMPUTED","FINAL_OUTLIER_SAITS_IMPUTED"]),
#                            on=["DEVICE_ID", "DATE"], how="left")

In [ ]:
merged_df.head()

#### Nullify the outliers

In [ ]:
imputed_columns = [
    "MEAN_IMPUTED", "MEDIAN_IMPUTED", "FFILL_IMPUTED", "BFILL_IMPUTED",
    "LINEAR_IMPUTED", "CUBIC_IMPUTED", "KNN_IMPUTED",
    "LIN_REG_IMPUTED", # "SAITS_IMPUTED"
]

In [ ]:
# Define the validity check expressions for each daily diff
nullify = [
    pl.when(
        pl.col(f"FINAL_OUTLIER_{col}")
    )
    .then(None)
    .otherwise(pl.col(col))
    .alias(col)
    for col in imputed_columns
]

new_df = merged_df.with_columns(nullify)

## Dimensions Scoring

In [ ]:
new_df.head()

### Accuracy

Compute accuracy for each device and the whole dataset

In [ ]:
# Ensure unique column names for each DataFrame
accuracy_results = []

for col in imputed_columns:
    # Define the outlier column for this imputed column
    outlier_column = f"FINAL_OUTLIER_{col}"

    # Create a new column for accuracy (1 for accurate, 0 for outlier)
    accuracy_col = f"ACCURACY_{col}"
    df = new_df.with_columns(
        (pl.col(outlier_column) == False).cast(pl.Int8).alias(accuracy_col)  # Accurate = 1, Outlier = 0
    )

    # Compute device-level accuracy
    accuracy_df = df.group_by("DEVICE_ID").agg([
        pl.sum(accuracy_col).alias(f"ACCURATE_RECORDS_{col}"),
        pl.count(accuracy_col).alias(f"TOTAL_RECORDS_{col}"),
        (pl.sum(accuracy_col) / pl.count(accuracy_col) * 100).alias(f"ACCURACY_PERCENT_{col}")
    ])

    # Rename DEVICE_ID to make it unique in each DataFrame
    accuracy_df = accuracy_df.rename({"DEVICE_ID": f"DEVICE_ID_{col}"})

    # Append the result to the list
    accuracy_results.append(accuracy_df)

# Combine all results horizontally
final_accuracy_df = pl.concat(accuracy_results, how="horizontal")

# Optionally restore the original DEVICE_ID column
final_accuracy_df = final_accuracy_df.with_columns(
    pl.col(f"DEVICE_ID_{imputed_columns[0]}").alias("DEVICE_ID")
).drop([f"DEVICE_ID_{col}" for col in imputed_columns])

In [ ]:
final_accuracy_df.head()

In [ ]:
# Select only DEVICE_ID and the DQS_FINAL columns for each imputation
final_accuracy_clean_df = final_accuracy_df.select(
    ["DEVICE_ID"] + [f"ACCURACY_PERCENT_{col}" for col in imputed_columns]
)

# Display the filtered DataFrame
final_accuracy_clean_df.head()

### Validity

In [ ]:
# Placeholder for validity results
validity_results = []

for col in imputed_columns:
    # Define the validity column for this imputed column
    validity_column = f"VALIDITY_{col}"

    # Compute device-level validity metrics
    validity_df = new_df.group_by("DEVICE_ID").agg([
        pl.sum(validity_column).alias(f"VALID_RECORDS_{col}"),  # Count valid records
        pl.count(validity_column).alias(f"TOTAL_RECORDS_{col}"),  # Total records
        (pl.sum(validity_column) / pl.count(validity_column) * 100).alias(f"VALIDITY_PERCENT_{col}")  # Validity percentage
    ])

    # Rename DEVICE_ID to make it unique in each DataFrame
    validity_df = validity_df.rename({"DEVICE_ID": f"DEVICE_ID_{col}"})

    # Append the result to the list
    validity_results.append(validity_df)

# Combine all results horizontally
final_validity_df = pl.concat(validity_results, how="horizontal")

# Restore the original DEVICE_ID column
final_validity_df = final_validity_df.with_columns(
    pl.col(f"DEVICE_ID_{imputed_columns[0]}").alias("DEVICE_ID")
).drop([f"DEVICE_ID_{col}" for col in imputed_columns])

In [ ]:
final_validity_df.head()

In [ ]:
# Select only DEVICE_ID and the DQS_FINAL columns for each imputation
final_validity_clean_df = final_validity_df.select(
    ["DEVICE_ID"] + [f"VALIDITY_PERCENT_{col}" for col in imputed_columns]
)

# Display the filtered DataFrame
final_validity_clean_df.head()

### Completeness

In [ ]:
# Placeholder for completeness results
completeness_results = []

for col in imputed_columns:
    # Define the outlier column for this imputed column
    outlier_column = f"FINAL_OUTLIER_{col}"

    # Step 1: Compute REPORTED_DAYS and EXPECTED_DAYS for each device
    completeness_df = new_df.with_columns(
        (pl.col(outlier_column) == False).cast(pl.Int8).alias(f"VALID_DAYS_{col}")  # Valid days column
    ).group_by("DEVICE_ID").agg([
        # Sum up the valid days
        pl.sum(f"VALID_DAYS_{col}").alias(f"REPORTED_DAYS_{col}"),
        # Calculate the total expected days based on the date range
         (((pl.col("DATE").max() - pl.col("DATE").min()).cast(pl.Int64)/ 86400000) + 1)
        .alias(f"EXPECTED_DAYS_{col}")
    ])

    # Step 2: Compute Completeness Percentage for each device
    completeness_df = completeness_df.with_columns(
        ((pl.col(f"REPORTED_DAYS_{col}") / pl.col(f"EXPECTED_DAYS_{col}")) * 100).alias(f"COMPLETENESS_PERCENT_{col}")
    )

    # Rename DEVICE_ID to make it unique in each DataFrame
    completeness_df = completeness_df.rename({"DEVICE_ID": f"DEVICE_ID_{col}"})

    # Append the result to the list
    completeness_results.append(completeness_df)

# Combine all results horizontally
final_completeness_df = pl.concat(completeness_results, how="horizontal")

# Restore the original DEVICE_ID column
final_completeness_df = final_completeness_df.with_columns(
    pl.col(f"DEVICE_ID_{imputed_columns[0]}").alias("DEVICE_ID")
).drop([f"DEVICE_ID_{col}" for col in imputed_columns])


In [ ]:
final_completeness_df.head()

In [ ]:
# Select only DEVICE_ID and the DQS_FINAL columns for each imputation
final_completeness_clean_df = final_completeness_df.select(
    ["DEVICE_ID"] + [f"COMPLETENESS_PERCENT_{col}" for col in imputed_columns]
)

# Display the filtered DataFrame
final_completeness_clean_df.head()

### Timeliness

Load the initial data to access the previous metrics. Timeliness is constant, so it's needed to recompute DQS post imputation.

In [ ]:
metrics_df = pl.read_csv('../exports/devices_to_clean.csv', infer_schema_length=100000)

In [ ]:
metrics_df.head()
# keep a copy
metrics_df_copy = metrics_df

In [ ]:
timeliness_df = metrics_df.select(['DEVICE_ID','AVG_TIMELINESS',])

In [ ]:
timeliness_df.head()

### Overall Data Quality Score

In [ ]:
# Placeholder for DQS results across imputed columns
dqs_results = []

# Timeliness metrics (does not change across imputations)
timeliness_metrics = timeliness_df

# AHP weights - From Data Quality Assessment
ahp_weights = {
    "timeliness": 0.054102,
    "validity": 0.310792,
    "completeness": 0.317553,
    "accuracy": 0.317553
}

equal_weight = 0.5
ahp_weight = 0.5

# Step 1: Iterate over imputed columns and compute DQS for each
for col in imputed_columns:
    # Combine Completeness, Accuracy, Validity for the current imputed column
    metrics_df = final_completeness_df.select([
        "DEVICE_ID",
        f"COMPLETENESS_PERCENT_{col}"
    ]).join(
        final_accuracy_df.select(["DEVICE_ID", f"ACCURACY_PERCENT_{col}"]),
        on="DEVICE_ID"
    ).join(
        final_validity_df.select(["DEVICE_ID", f"VALIDITY_PERCENT_{col}"]),
        on="DEVICE_ID"
    ).join(
        timeliness_metrics,
        on="DEVICE_ID" 
    )

    # Step 2: Compute Equal Weighting DQS
    metrics_df = metrics_df.with_columns(
        (
            0.25 * pl.col(f"COMPLETENESS_PERCENT_{col}") +
            0.25 * pl.col(f"ACCURACY_PERCENT_{col}") +
            0.25 * pl.col("AVG_TIMELINESS") +
            0.25 * pl.col(f"VALIDITY_PERCENT_{col}")
        ).alias(f"DQS_EQUAL_{col}")
    )

    # Step 3: Compute AHP Weighting DQS
    metrics_df = metrics_df.with_columns(
        (
            ahp_weights["accuracy"] * pl.col(f"ACCURACY_PERCENT_{col}") +
            ahp_weights["completeness"] * pl.col(f"COMPLETENESS_PERCENT_{col}") +
            ahp_weights["timeliness"] * pl.col("AVG_TIMELINESS") +
            ahp_weights["validity"] * pl.col(f"VALIDITY_PERCENT_{col}")
        ).alias(f"DQS_AHP_{col}")
    )

    # Step 4: Combine Equal and AHP Weighting (Final DQS)
    metrics_df = metrics_df.with_columns(
        (
            (pl.col(f"DQS_EQUAL_{col}") * equal_weight) +
            (pl.col(f"DQS_AHP_{col}") * ahp_weight)
        ).alias(f"DQS_FINAL_{col}")
    )

    # Rename all columns (including DEVICE_ID and AVG_TIMELINESS) to make them unique for this imputation column
    metrics_df = metrics_df.rename({
        "DEVICE_ID": f"DEVICE_ID_{col}",
        "AVG_TIMELINESS": f"AVG_TIMELINESS_{col}"
    })

    # Append the result for this imputed column
    dqs_results.append(metrics_df)

# Step 5: Combine DQS for all imputed columns horizontally
final_dqs_df = pl.concat(dqs_results, how="horizontal")

In [ ]:
final_dqs_df.head()

In [ ]:
final_dqs_df = final_dqs_df.rename({"DEVICE_ID_MEAN_IMPUTED":"DEVICE_ID"})

In [ ]:
# Select only DEVICE_ID and the DQS_FINAL columns for each imputation
final_dqs_filtered_df = final_dqs_df.select(
    ["DEVICE_ID",] + [f"DQS_FINAL_{col}" for col in imputed_columns]
)

# Display the filtered DataFrame
final_dqs_filtered_df.head()

In [ ]:
final_dqs_filtered_df['DEVICE_ID'].n_unique()

## Visualization

In [ ]:
final_dqs_filtered_df.write_csv('../exports/final_dqs.csv')

In [ ]:
final_dqs_filtered_df_pd = final_dqs_filtered_df.to_pandas()

In [ ]:
# Ensure the output directory exists
output_dir = "../visualization/plots"

# Plot 1: Bar Chart of DQS for Each Imputation Method per Device
plt.figure(figsize=(18, 6))
final_dqs_filtered_df_pd.set_index("DEVICE_ID").plot(kind="bar", figsize=(18, 6))
plt.xlabel("Device ID")
plt.ylabel("Final DQS (%)")
plt.title("Final Data Quality Score for Each Imputation Method per Device")
plt.xticks(rotation=90)
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.legend(title="Imputation Methods", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.savefig(f"{output_dir}/final_dqs_per_device.png", dpi=300, bbox_inches="tight")
plt.show()

In [ ]:
# Plot 2: Boxplot of DQS Scores Across All Imputation Methods
plt.figure(figsize=(12, 6))
sns.boxplot(data=final_dqs_filtered_df_pd.drop(columns=["DEVICE_ID"]))
plt.xlabel("Imputation Methods")
plt.ylabel("Final DQS (%)")
plt.title("Boxplot of DQS Across Different Imputation Methods")
plt.xticks(rotation=45)
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.savefig(f"{output_dir}/final_dqs_boxplot.png", dpi=300, bbox_inches="tight")
plt.show()

In [ ]:
# Plot 3: Histogram of DQS Distribution Across Imputation Methods
plt.figure(figsize=(12, 6))
final_dqs_filtered_df_pd.drop(columns=["DEVICE_ID"]).plot(kind="hist", alpha=0.5, bins=15, figsize=(12, 6))
plt.xlabel("Final DQS (%)")
plt.ylabel("Frequency")
plt.title("Distribution of Final DQS Across Different Imputation Methods")
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.legend(title="Imputation Methods", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.savefig(f"{output_dir}/final_dqs_histogram.png", dpi=300, bbox_inches="tight")
plt.show()

In [ ]:
# Plot 4: Heatmap of DQS Scores for Visual Comparison Across Methods (Reversed Color)
plt.figure(figsize=(12, 8))
sns.heatmap(
    final_dqs_filtered_df_pd.set_index("DEVICE_ID"), 
    annot=True, cmap="RdYlGn", fmt=".1f", linewidths=0.5
)
plt.xlabel("Imputation Methods")
plt.ylabel("Device ID")
plt.title("Heatmap of Final Data Quality Scores Across Imputation Methods")
plt.savefig(f"{output_dir}/final_dqs_heatmap.png", dpi=300, bbox_inches="tight")
plt.show()

In [ ]:
# Define groups
univariates = ["DQS_FINAL_FFILL_IMPUTED", "DQS_FINAL_BFILL_IMPUTED","DQS_FINAL_MEAN_IMPUTED", 
               "DQS_FINAL_MEDIAN_IMPUTED", "DQS_FINAL_LINEAR_IMPUTED", "DQS_FINAL_CUBIC_IMPUTED"]

multivariates = ["DQS_FINAL_KNN_IMPUTED","DQS_FINAL_LIN_REG_IMPUTED", ]  #"DQS_FINAL_SAITS_IMPUTED"

# Compute average DQS per category
final_dqs_filtered_df_pd["DQS_UNIVARIATES"] = final_dqs_filtered_df_pd[univariates].mean(axis=1)
final_dqs_filtered_df_pd["DQS_MULTIVARIATES"] = final_dqs_filtered_df_pd[multivariates].mean(axis=1)

# Compute overall average DQS for each category
avg_dqs_per_group = {
    "Univariate Methods": final_dqs_filtered_df_pd["DQS_UNIVARIATES"].mean(),
    "Multivariate Methods": final_dqs_filtered_df_pd["DQS_MULTIVARIATES"].mean(),

}
print(avg_dqs_per_group)

In [ ]:
# Convert to DataFrame for plotting
avg_dqs_df = pd.DataFrame.from_dict(avg_dqs_per_group, orient="index", columns=["Average DQS"])

# Bar Chart: Comparison of Imputation Groups
plt.figure(figsize=(8, 6))
sns.barplot(x=avg_dqs_df.index, y=avg_dqs_df["Average DQS"], palette=["blue", "green", "red"])
plt.ylabel("Average Final DQS (%)")
plt.xlabel("Imputation Category")
plt.title("Comparison of Data Quality Scores Across Imputation Techniques")
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.ylim(75, 100)  # Adjust as needed
plt.savefig(f"{output_dir}/imputation_category_comparison.png", dpi=300, bbox_inches="tight")
plt.show()

# Boxplot: Distribution of DQS within Each Group
plt.figure(figsize=(10, 6))
sns.boxplot(data=final_dqs_filtered_df_pd[["DQS_UNIVARIATES", "DQS_MULTIVARIATES",]])
plt.xlabel("Imputation Categories")
plt.ylabel("Final DQS (%)")
plt.title("Distribution of Data Quality Scores Across Imputation Methods")
plt.xticks(ticks=[0, 1], labels=["Univariates Methods", "Multivariates Methods",])
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.savefig(f"{output_dir}/imputation_category_boxplot.png", dpi=300, bbox_inches="tight")
plt.show()


In [ ]:
avg_dqs_df.head()

In [ ]:
avg_dqs_df.to_csv('../exports/avg_dqs_method.csv')

#### Pre and Post Imputation Comparison

In [ ]:
metrics_df_copy.head()

In [ ]:
# metrics_df = metrics_df.rename({"DEVICE_ID_SAITS_IMPUTED":"DEVICE_ID"})

# # Ensure metrics_df_copy contains the required columns
# source_df = metrics_df_copy.select(["DEVICE_ID", "DQS_FINAL_DEVICE"])

# # Perform the join (left join ensures all DEVICE_IDs in metrics_df are retained)
# metrics_df = metrics_df.join(source_df, on="DEVICE_ID", how="left")

metrics_df_pd = metrics_df_copy.to_pandas()

# Merge previous and post-imputation DQS data
comparison_df = final_dqs_filtered_df_pd.merge(metrics_df_pd, on="DEVICE_ID", suffixes=("_IMPUTED", "_PRE"))

In [ ]:
# Compute DQS improvement for each imputation method
for col in [
    "DQS_FINAL_MEAN_IMPUTED", "DQS_FINAL_MEDIAN_IMPUTED", "DQS_FINAL_FFILL_IMPUTED", 
    "DQS_FINAL_BFILL_IMPUTED", "DQS_FINAL_LINEAR_IMPUTED", "DQS_FINAL_CUBIC_IMPUTED",
    "DQS_FINAL_KNN_IMPUTED","DQS_FINAL_LIN_REG_IMPUTED" 
]:
    #, "DQS_FINAL_SAITS_IMPUTED"
    comparison_df[f"{col}_CHANGE"] = comparison_df[col] - comparison_df["DQS_FINAL_DEVICE"]

# Identify best method per device (the one with highest DQS increase)
comparison_df["BEST_METHOD"] = comparison_df[
    [
        "DQS_FINAL_MEAN_IMPUTED", "DQS_FINAL_MEDIAN_IMPUTED", "DQS_FINAL_FFILL_IMPUTED", 
        "DQS_FINAL_BFILL_IMPUTED", "DQS_FINAL_LINEAR_IMPUTED", "DQS_FINAL_CUBIC_IMPUTED",
        "DQS_FINAL_KNN_IMPUTED", "DQS_FINAL_LIN_REG_IMPUTED"
    ]
#, "DQS_FINAL_SAITS_IMPUTED"
].idxmax(axis=1)

# Save comparison results
comparison_df.to_csv("../exports/dqs_comparison.csv", index=False)

# 1. Boxplot: DQS Improvement per Imputation Method
plt.figure(figsize=(12, 6))
dqs_change_cols = [col for col in comparison_df.columns if "_CHANGE" in col]
sns.boxplot(data=comparison_df[dqs_change_cols])
plt.xticks(ticks=range(len(dqs_change_cols)), labels=[col.replace("_CHANGE", "").replace("DQS_FINAL_", "") for col in dqs_change_cols], rotation=45)
plt.xlabel("Imputation Method")
plt.ylabel("DQS Improvement")
plt.title("Impact of Imputation Techniques on Data Quality Score")
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.savefig(f"{output_dir}/dqs_improvement_boxplot.png", dpi=300, bbox_inches="tight")
plt.show()

In [ ]:
comparison_df.to_csv('../exports/comparison_df.csv')

In [ ]:
# 2. Bar Chart: Average DQS Improvement per Imputation Method
avg_dqs_improvement = comparison_df[dqs_change_cols].mean().sort_values(ascending=False)

plt.figure(figsize=(12, 6))
sns.barplot(x=avg_dqs_improvement.index.str.replace("_CHANGE", "").str.replace("DQS_FINAL_", ""), y=avg_dqs_improvement.values, palette="coolwarm_r")
plt.xlabel("Imputation Method")
plt.ylabel("Average DQS Improvement")
plt.title("Average DQS Improvement Across Imputation Methods")
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.xticks(rotation=45)
plt.savefig(f"{output_dir}/dqs_improvement_barplot.png", dpi=300, bbox_inches="tight")
plt.show()

In [ ]:
avg_dqs_improvement

In [ ]:
# 3. Scatter Plot: Pre-Imputation vs. Best Post-Imputation DQS
plt.figure(figsize=(8, 6))
sns.scatterplot(x=comparison_df["DQS_FINAL_DEVICE"], y=comparison_df[dqs_change_cols].max(axis=1), alpha=0.7, color="purple")
plt.xlabel("Pre-Imputation DQS")
plt.ylabel("Best Post-Imputation DQS")
plt.title("Pre-Imputation vs. Best Post-Imputation Data Quality Score")
plt.grid(True, linestyle="--", alpha=0.7)
plt.savefig(f"{output_dir}/dqs_scatter_best.png", dpi=300, bbox_inches="tight")
plt.show()


In [ ]:
# 4. Countplot: Best Performing Imputation Method for Each Device
plt.figure(figsize=(12, 6))
sns.countplot(y=comparison_df["BEST_METHOD"], order=comparison_df["BEST_METHOD"].value_counts().index, palette="coolwarm")
plt.xlabel("Number of Devices")
plt.ylabel("Best Performing Imputation Method")
plt.title("Best Imputation Method for Each Device")
plt.grid(axis="x", linestyle="--", alpha=0.7)
plt.savefig(f"{output_dir}/best_imputation_method_countplot.png", dpi=300, bbox_inches="tight")
plt.show()
